In [ ]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [ ]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

# בדיקות לקוד נומרי בפיזיקה

במחברת זו נתרכז בבדיקת **פונקציות נומריות פיזיקליות**,  
כאלו שעוסקות בחישובים עשרוניים, יחידות מידה, ורעש ניסויי.  
נלמד כיצד לכתוב בדיקות מדויקות אך גם סלחניות לשגיאות חישוב קטנות —  
שכיחות מאוד במדעי הפיזיקה.

## מטרות הלמידה
- להשתמש ב־`assertAlmostEqual` לבדיקת מספרים צפים.  
- לבדוק פונקציות עם טולרנסים נומריים (סבילות חישובית).  
- לבדוק קלטים לא חוקיים (כמו ערכים שליליים או יחידות שגויות).  
- לבדוק המרות יחידות, פונקציות אנרגיה, ושגיאות רנדומליות בצורה מבוקרת.

## בדיקה נומרית פשוטה עם assertAlmostEqual
הפונקציה `kinetic_energy(mass, velocity)` מחשבת את האנרגיה הקינטית של גוף לפי הנוסחה הפיזיקלית:
$$
E_k = \frac{1}{2} m v^2
$$

- אם המסה שלילית — נזרקת חריגה (`ValueError`), כי מבחינה פיזיקלית אין משמעות למסה שלילית.  
- הבדיקות בודקות שלושה מצבים:
  1. חישובים רגילים עם ערכים חיוביים.
  2. מקרים שבהם המסה או המהירות אפסיים (האנרגיה אמורה להיות 0).
  3. מקרה חריג שבו המסה שלילית (אמור להיזרק חריג).

זהו תרגיל קלאסי לשימוש ב־`assertAlmostEqual` עם חישובים עשרוניים ולבדיקות תקינות קלטים פיזיקליים.

In [ ]:
import unittest

# --- TODO: Complete the function ---
def kinetic_energy(mass, velocity):
    """Compute kinetic energy (Joules)."""
    # TODO: if mass is negative → raise ValueError
    # TODO: return 0.5 * mass * velocity**2
    pass


class TestKineticEnergy(unittest.TestCase):
    def test_basic_values(self):
        """TODO: Check typical kinetic energy values."""
        # TODO: verify that kinetic_energy(2.0, 3.0) ≈ 9.0
        # TODO: verify that kinetic_energy(1.0, 10.0) ≈ 50.0
        pass

    def test_zero_mass_or_velocity(self):
        """TODO: Energy should be zero if mass or velocity is zero."""
        # TODO: test kinetic_energy(0, 5) == 0
        # TODO: test kinetic_energy(2, 0) == 0
        pass

    def test_negative_mass_raises(self):
        """TODO: Negative mass should raise ValueError."""
        # TODO: use assertRaises(ValueError) for negative mass
        pass


# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestKineticEnergy)
unittest.TextTestRunner(verbosity=2).run(suite)


`````{admonition} פתרון
:class: dropdown, tip
```python
def kinetic_energy(mass, velocity):
    """Compute kinetic energy (Joules)."""
    if mass < 0:
        raise ValueError("Mass must be non-negative")
    return 0.5 * mass * velocity**2

class TestKineticEnergy(unittest.TestCase):
    def test_basic_values(self):
        """Check typical kinetic energy values."""
        self.assertAlmostEqual(kinetic_energy(2.0, 3.0), 9.0, places=6)
        self.assertAlmostEqual(kinetic_energy(1.0, 10.0), 50.0, places=6)

    def test_zero_mass_or_velocity(self):
        """Energy should be zero if mass or velocity is zero."""
        self.assertEqual(kinetic_energy(0, 5), 0)
        self.assertEqual(kinetic_energy(2, 0), 0)

    def test_negative_mass_raises(self):
        """Negative mass should raise ValueError."""
        with self.assertRaises(ValueError):
            kinetic_energy(-1.0, 5.0)
```
`````

```{note}
כאן אנחנו נדרשים לבדוק גם את הדיוק וגם את הקלטים הבלתי תקינים — מצב קלאסי בפיזיקה ניסויית.
```

## בדיקות טולרנס (Tolerance) לערכים מחושבים
הפונקציה `estimate_gravity(L, T)` מחשבת את תאוצת הכובד  $g$  על סמך ניסוי מטוטלת:  

$$
g = \frac{4 \pi^2 L}{T^2}
$$

כאשר:
-  $L$  — אורך המטוטלת (במטרים).  
-  $T$  — זמן המחזור (בשניות).  

הפונקציה מעלה חריגה (`ValueError`) אם אחד מהערכים לא חיובי,  
והבדיקות מוודאות שהתוצאה קרובה לערך הצפוי (9.81 m/s²) עם סבילות (`delta`) קטנה —  
כפי שנדרש בבדיקות ניסוי אמפיריות שבהן יש רעש מדידה קטן.

בדיקה נוספת מוודאת שהתוכנית מגיבה נכון לקלטים לא תקינים כמו אפס או ערכים שליליים.

In [ ]:
import unittest
import math

# --- TODO: Complete the function ---
def estimate_gravity(L, T):
    """Estimate g using pendulum formula."""
    # TODO: if L <= 0 or T <= 0 → raise ValueError
    # TODO: compute and return 4 * math.pi**2 * L / (T**2)
    pass


class TestGravityEstimate(unittest.TestCase):
    def test_gravity_approximation(self):
        """TODO: Check g estimate within tolerance."""
        # Arrange
        # TODO: set L=1.0, T=2.006, g_expected=9.81
        # Act
        # TODO: call estimate_gravity(L, T)
        # Assert
        # TODO: use assertAlmostEqual with delta=0.05
        pass

    def test_invalid_values_raise(self):
        """TODO: Zero or negative L/T should raise ValueError."""
        # TODO: use assertRaises(ValueError) for estimate_gravity(0, 2)
        # TODO: use assertRaises(ValueError) for estimate_gravity(1, -3)
        pass


# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestGravityEstimate)
unittest.TextTestRunner(verbosity=2).run(suite)


`````{admonition} פתרון
:class: dropdown, tip
```python
def estimate_gravity(L, T):
    """Estimate g using pendulum formula."""
    if L <= 0 or T <= 0:
        raise ValueError("L and T must be positive")
    return 4 * math.pi**2 * L / (T**2)


class TestGravityEstimate(unittest.TestCase):
    def test_gravity_approximation(self):
        """Check g estimate within tolerance."""
        L, T = 1.0, 2.006  # realistic lab values
        g_expected = 9.81
        g_calc = estimate_gravity(L, T)
        # allow small difference due to measurement precision
        self.assertAlmostEqual(g_calc, g_expected, delta=0.05)

    def test_invalid_values_raise(self):
        """Zero or negative L/T should raise ValueError."""
        with self.assertRaises(ValueError):
            estimate_gravity(0, 2)
        with self.assertRaises(ValueError):
            estimate_gravity(1, -3)
```
`````

```{note}
שימוש ב־`delta` מאפשר לבדוק טווח טולרנס — חשוב כאשר הערכים מגיעים ממדידות ניסוי אמיתיות.
```

## ניהול רעש רנדומלי בבדיקות
הפונקציה `noisy_measurement(true_value, noise_level=0.1)` מדמה מדידה פיזיקלית עם **רעש אקראי** —  
כלומר, הערך האמיתי $ v_{\text{true}} $ מקבל סטייה מקרית בתחום ±`noise_level`.

- אם רמת הרעש (`noise_level`) שלילית, נזרקת חריגה (`ValueError`),  
  מפני שלא ניתן להגדיר טווח רעש שלילי.  
- הבדיקות מוודאות שהמדידה האקראית **תישאר בגבולות הפיזיקליים הסבירים**,  
  כלומר לא רחוקה יותר מ־±`noise_level` מהערך האמיתי.

בנוסף נבדק שקריאה עם `noise_level` שלילי גורמת לחריגה —  
כדי לוודא טיפול נכון בקלטים לא תקינים.

In [ ]:
import unittest
import random

# --- TODO: Complete the function ---
def noisy_measurement(true_value, noise_level=0.1):
    """Return a value with random uniform noise ±noise_level."""
    # TODO: if noise_level < 0 → raise ValueError
    # TODO: return true_value + random.uniform(-noise_level, noise_level)
    pass


class TestNoisyMeasurement(unittest.TestCase):
    def test_value_within_bounds(self):
        """TODO: Check that noisy output stays within expected range."""
        # Arrange
        true = 10.0
        noise = 0.5
        # Act + Assert
        # TODO: repeat the measurement multiple times (e.g., 100 iterations)
        # TODO: use assertTrue to ensure each value is within [true - noise, true + noise]
        pass

    def test_negative_noise_raises(self):
        """TODO: Ensure that negative noise level raises ValueError."""
        # TODO: use assertRaises(ValueError) for noisy_measurement(10, -0.1)
        pass


# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestNoisyMeasurement)
unittest.TextTestRunner(verbosity=2).run(suite)

`````{admonition} פתרון
:class: dropdown, tip
```python
def noisy_measurement(true_value, noise_level=0.1):
    """Return a value with random uniform noise ±noise_level."""
    if noise_level < 0:
        raise ValueError("Noise level must be non-negative")
    return true_value + random.uniform(-noise_level, noise_level)


class TestNoisyMeasurement(unittest.TestCase):
    def test_value_within_bounds(self):
        """Check that noisy output stays within expected range."""
        true = 10.0
        noise = 0.5
        for _ in range(100):
            val = noisy_measurement(true, noise)
            self.assertTrue(9.5 <= val <= 10.5)

    def test_negative_noise_raises(self):
        with self.assertRaises(ValueError):
            noisy_measurement(10, -0.1)
```
`````

```{note}
אין טעם לבדוק ערך מדויק — במקום זאת נבדוק שהערך נשאר **בתחום פיזיקלי סביר**.
```

## 4️⃣ תרגיל לסטודנטים — בדיקת המרת יחידות אנרגיה
הפונקציה `joules_to_calories(joules)` ממירה אנרגיה מג'ול (J) לקלוריות (cal)  
על פי הקשר הפיזיקלי:

$$
1 \text{ cal} = 4.184 \text{ J}
$$

כלומר:
$$
\text{calories} = \frac{\text{joules}}{4.184}
$$

- אם הערך שלילי — נזרקת חריגה (`ValueError`), כי אנרגיה שלילית אינה תקפה בהקשר זה.  
- הבדיקות בודקות:
  1. המרה בסיסית מדויקת של ערך אחד.  
  2. סדרת ערכים שונים בעזרת `subTest` (בדיקה פולימורפית ונקייה).  
  3. חריגה על קלט שלילי.

זהו תרגיל טיפוסי בבדיקות המרות יחידות,  
שבו נבדקת גם **נכונות נומרית** וגם **בדיקת תקינות קלטים פיזיקליים**.


In [ ]:
import unittest

def joules_to_calories(joules):
    """TODO: Convert Joules to Calories. Validate non-negative input."""
    # TODO: if joules < 0 → raise ValueError
    # TODO: return joules / 4.184
    pass


class TestEnergyConversion(unittest.TestCase):
    def test_basic_conversion(self):
        """TODO: Check conversion accuracy."""
        # TODO: check that joules_to_calories(4.184) == 1.0 (use assertAlmostEqual)
        pass

    def test_multiple_values(self):
        """TODO: Use subTest to check several values."""
        # TODO: create test cases: [(0,0), (418.4,100), (8.368,2)]
        # TODO: loop with subTest(joules=...)
        # TODO: compare using assertAlmostEqual with places=6
        pass

    def test_negative_input_raises(self):
        """TODO: Verify that negative input raises ValueError."""
        # TODO: use assertRaises(ValueError)
        pass


# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestEnergyConversion)
unittest.TextTestRunner(verbosity=2).run(suite)

`````{admonition} פתרון
:class: dropdown, tip
```python
def joules_to_calories(joules):
    """Convert Joules to Calories. Validate non-negative input."""
    if joules < 0:
        raise ValueError("Energy cannot be negative")
    return joules / 4.184


class TestEnergyConversion(unittest.TestCase):
    def test_basic_conversion(self):
        """Check conversion accuracy."""
        self.assertAlmostEqual(joules_to_calories(4.184), 1.0, places=6)

    def test_multiple_values(self):
        """Use subTest to check several values."""
        test_cases = [
            (0, 0),
            (418.4, 100),
            (8.368, 2)
        ]
        for joules, expected in test_cases:
            with self.subTest(joules=joules):
                self.assertAlmostEqual(joules_to_calories(joules), expected, places=6)

    def test_negative_input_raises(self):
        """Verify that negative input raises ValueError."""
        with self.assertRaises(ValueError):
            joules_to_calories(-10.0)
```
`````

## 5️⃣ תרגיל נוסף — חישוב אנרגיה פוטנציאלית כבידתית

## 💡 הסבר על הקוד

הפונקציה `potential_energy(m, g, h)` מחשבת את האנרגיה הפוטנציאלית הכבידתית לפי הנוסחה הפיזיקלית:

$$
E_p = mgh
$$

כאשר:
-  $m$  — מסה (בק"ג)  
-  $g$  — תאוצת הכובד (ב־m/s²)  
-  $h$  — גובה (במטרים)

אם אחד מהערכים שלילי, נזרקת חריגה (`ValueError`),  
כי במודל הפיזיקלי הזה לא נבחנות מסות, תאוצות או גבהים שליליים.

הבדיקות בודקות:
1. חישוב תקין לערכים רגילים.  
2. תוצאה אפסית לגובה אפס.  
3. זריקת חריגה עבור ערכים שליליים.


In [ ]:
import unittest

def potential_energy(m, g, h):
    """TODO: Compute gravitational potential energy (E = mgh)."""
    # TODO: validate m, g, h are non-negative
    # TODO: return m * g * h
    pass


class TestPotentialEnergy(unittest.TestCase):
    def test_physical_values(self):
        """TODO: Check correct computation for typical values."""
        # TODO: verify that potential_energy(2, 9.81, 10) == 196.2 (use assertAlmostEqual)
        pass

    def test_zero_height(self):
        """TODO: Height 0 should give energy 0."""
        # TODO: verify potential_energy(1, 9.81, 0) == 0
        pass

    def test_negative_values_raise(self):
        """TODO: Negative mass, g, or height should raise ValueError."""
        # TODO: use assertRaises(ValueError) for negative inputs
        pass


# --- Run tests in Jupyter-friendly mode ---
suite = unittest.TestLoader().loadTestsFromTestCase(TestPotentialEnergy)
unittest.TextTestRunner(verbosity=2).run(suite)

`````{admonition} פתרון
:class: dropdown, tip
```python
def potential_energy(m, g, h):
    """Compute gravitational potential energy (E = mgh)."""
    if m < 0 or g < 0 or h < 0:
        raise ValueError("Mass, gravity, and height must be non-negative")
    return m * g * h


class TestPotentialEnergy(unittest.TestCase):
    def test_physical_values(self):
        """Check correct computation for typical values."""
        # Arrange + Act + Assert
        self.assertAlmostEqual(potential_energy(2, 9.81, 10), 196.2, places=3)

    def test_zero_height(self):
        """Height 0 should give energy 0."""
        self.assertEqual(potential_energy(1, 9.81, 0), 0)

    def test_negative_values_raise(self):
        """Negative mass, g, or height should raise ValueError."""
        with self.assertRaises(ValueError):
            potential_energy(-1, 9.81, 10)
        with self.assertRaises(ValueError):
            potential_energy(2, -9.81, 10)
        with self.assertRaises(ValueError):
            potential_energy(1, 9.81, -5)
```
`````

```{note}
זוהי דוגמה קלאסית לבדיקת חישוב פיזיקלי פשוט עם טיפול בחריגות —  
שילוב בין מתמטיקה מדויקת לולידציה ריאלית של נתונים.
```

## סיכום
במחברת זו:
- למדנו לבדוק **קוד נומרי** תוך שימוש ב־`assertAlmostEqual` ו־`delta` לטולרנס.  
- עסקנו בבעיות פיזיקליות נפוצות: אנרגיה, כבידה, רעש מדידה, והמרות יחידות.  
- הבנו איך לבדוק **ערכים בקצוות**, וגם **תחומים פיזיקליים סבירים** במקום שוויון מדויק.

```{note}
זוהי מיומנות חשובה לכל פיזיקאי־מתכנת: לדעת מתי “שגיאה קטנה” היא תקינה,  
ומתי היא סימן לחישוב או קלט שגוי.
```